In [ ]:
from transformers import AutoTokenizer,DataCollatorWithPadding
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:

!pip install transformers


     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 51.9 MB/s 
     |████████████████████████████████| 895 kB 46.0 MB/s 
     |████████████████████████████████| 3.3 MB 33.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from datasets import load_dataset
from datasets import get_dataset_config_names

In [ ]:
dataset = load_dataset('imdb', 'plain_text',split='train[:5%]+train[90%:100%]') #taking first 5% and last 10% of data. This is train data

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
dataset = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/voc

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
data = DataCollatorWithPadding(tokenizer)
#train_set.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [ ]:
model_name = "bert-base-cased"
model=BertForSequenceClassification.from_pretrained(model_name,num_labels =2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin from cache at /

In [ ]:
training_args = TrainingArguments(
    output_dir = ".",
    num_train_epochs=1,
    per_device_train_batch_size=8,  
    weight_decay=0.01,        
    learning_rate = 2e-5,      
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(model=model,args=training_args,train_dataset=dataset,tokenizer = tokenizer, data_collator=data)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 3750
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 469


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=469, training_loss=0.34784579734558235, metrics={'train_runtime': 723.1604, 'train_samples_per_second': 5.186, 'train_steps_per_second': 0.649, 'total_flos': 986666457600000.0, 'train_loss': 0.34784579734558235, 'epoch': 1.0})

In [ ]:
data = load_dataset('imdb', 'plain_text',split='test[97%:100%]+test[:2%]')#last 3 percent of dataset + first 3 percentof data this is test data
data = data.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
predictions = trainer.predict(data)
print(predictions.predictions.shape, predictions.label_ids.shape)
print(predictions.predictions,predictions.label_ids)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 1250
  Batch size = 8


(1250, 2) (1250,)
[[-2.737945    2.2292995 ]
 [-2.8789594   2.3584948 ]
 [-0.23778306 -0.02426953]
 ...
 [ 1.451404   -1.8772489 ]
 [ 1.9211419  -2.0044806 ]
 [ 1.8734185  -2.0328028 ]] [1 1 1 ... 0 0 0]


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)
accuracy_score(preds, predictions.label_ids) #accuracy score

0.8928